# Projet : Extraction et Recommandation de films et séries avec rdflib et SPARQL

## Objectif
L’objectif de ce mini-projet est l’extraction des données de la base de données DBPedia en utilisant le langage de requêtes SPARQL afin d'alimenter la base de données de séries que vous avez créé au premier TP, l’analyse de ces données et la recommandation de séries et de films.

# Description

Vous allez créer un système de recommandation de séries/films basé sur des données RDF qui stockent des informations sur les films, les utilisateurs et leurs préférences cinématographiques.

Les étapes du projet sont comme suit :

## 1.  Extraction des données

a)  Vous utiliserez Rdflib pour accéder aux données et vous effectuerez des requêtes SPARQL pour les extraire. Vous êtes libres dans le choix et la taille des données que vous allez extraire. Le jeu de données doit néanmoins être représentatif pour pouvoir fournir des recommandations précises. 
    
   - Les données à extraire : 
       - Films : Chaque film a un titre, un réalisateur, une année de sortie, un genre (e.g. : action, comédie, science-fiction), un résumé, une liste d'acteurs principaux, une durée, une évaluation du film, etc.
       - Réalisateur : Chaque réalisateur a un nom, une biographie et une liste de films qu'il a réalisés.
       - Acteurs : Chaque acteur a un nom, une biographie et une liste de films dans lesquels il a joué.
       - Genres :  Chaque genre a un nom et une description.
       - Utilisateurs : chaque utilisateur a un identifiant et des préférences cinématographiques (acteurs préférés, genres préférés, etc).
       - Évaluations : Elle est décrite par l'identifiant de l'utilisateur qui a donné l'évaluation, identifiant du film évalué la note attribuée au film, Commentaire ou avis sur le film.

   - Liens entre les entités : 

        - Les films sont associés à leurs acteurs, réalisateurs et genres.
        - Les utilisateurs sont associés aux films qu'ils ont évalués.
        - Les utilisateurs peuvent être liés entre eux en fonction de leurs préférences cinématographiques similaires.

   b)  Transformer les données en triplets RDF :  Vous allez transformer ces résultats en triplets RDF avant de les ajouter à votre graphe RDF existant. 
  
  c) Ajouter les données au graphe existant : Utilisez la méthode g.add() de votre graphe RDF, que vous avez créé au premier TP, pour ajouter les triplets RDF représentant les données DBpedia que vous avez transformées. 

In [74]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD

import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter

from sklearn.metrics import ndcg_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Create our custom namespace for film data
EX = Namespace("http://example.org/film#")

# =============================================================================
# 1. DATA EXTRACTION - SPARQL + RDF
# =============================================================================
def extract_data_sparql(endpoint, query):
    """
    Extracts data from a SPARQL endpoint and returns JSON bindings.
    """
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"]

def build_rdf_graph(data):
    """
    Builds and returns an RDF graph from the JSON results of a SPARQL query.
    """
    g = Graph()
    g.bind("ex", EX)
    
    for row in data:
        film_uri = URIRef(row["film"]["value"])
        
        # Title
        if "title" in row:
            g.add((film_uri, RDFS.label, Literal(row["title"]["value"], datatype=XSD.string)))
        # Director
        if "director" in row:
            director_uri = URIRef(row["director"]["value"])
            g.add((film_uri, EX.director, director_uri))
        # Actor
        if "actor" in row:
            actor_uri = URIRef(row["actor"]["value"])
            g.add((film_uri, EX.actor, actor_uri))
        # Genre
        if "genre" in row:
            genre_uri = URIRef(row["genre"]["value"])
            g.add((film_uri, EX.genre, genre_uri))
        # Release Date
        if "releaseDate" in row:
            g.add((film_uri, EX.releaseDate, Literal(row["releaseDate"]["value"], datatype=XSD.date)))
    return g

# Example DBpedia Query
endpoint = "http://dbpedia.org/sparql"
film_query = """
SELECT ?film ?title ?genre ?director ?releaseDate ?actor
WHERE {
  ?film rdf:type dbo:Film.
  ?film rdfs:label ?title.
  FILTER (lang(?title) = 'en').
  OPTIONAL { ?film dbo:genre ?genre. }
  OPTIONAL { ?film dbo:director ?director. }
  OPTIONAL { ?film dbo:releaseDate ?releaseDate. }
  OPTIONAL { ?film dbo:starring ?actor. }
}
LIMIT 700
"""
# Extract and build the graph
data = extract_data_sparql(endpoint, film_query)
graph = build_rdf_graph(data)

## 2. Prétraitement des données   

Nettoyez et traitez les données extraites pour supprimer les doublons, gérer les valeurs manquantes et normaliser.

In [75]:
# =============================================================================
# 2. DATA PREPROCESSING
# =============================================================================
def preprocess_graph(g):
    """
    Cleans and normalizes the RDF graph by removing duplicates,
    handling empty literals, and normalizing text to lowercase (if not date).
    """
    cleaned = Graph()
    cleaned.bind("ex", EX)
    
    for s, p, o in g:
        # Skip empty literals
        if isinstance(o, Literal) and not str(o).strip():
            continue
        
        # Normalize string literals (exclude date or numeric)
        if isinstance(o, Literal) and o.datatype != XSD.date:
            val = str(o).strip().lower()
            cleaned.add((s, p, Literal(val)))
        else:
            cleaned.add((s, p, o))
    return cleaned

graph = preprocess_graph(graph)


## 3. Analyse exploratoire des données 

- Créer des graphiques permettant de visualiser la distribution des films et séries dans votre base de données.
- Créer un graphique pour montrer les films et les séries les mieux notés
- Créer un nuage de points pour représenter la relation entre les caractéristiques

In [77]:
# =============================================================================
# 3. (Optional) EXPLORATORY DATA ANALYSIS
# =============================================================================
def visualize_data(g):
    """
    Visualize genre distribution and release year distribution.
    """
    # Genre distribution
    genres = []
    for s, p, o in g.triples((None, EX.genre, None)):
        genres.append(o.split("/")[-1])
    
    if genres:
        counter = Counter(genres)
        plt.figure(figsize=(8,5))
        plt.bar(counter.keys(), counter.values(), color='skyblue')
        plt.title("Genre Distribution")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print("No genre data to visualize.")
    
    # Release year distribution
    years = []
    for s, p, o in g.triples((None, EX.releaseDate, None)):
        try:
            year = int(str(o)[:4])  # parse the first 4 chars as year
            years.append(year)
        except:
            pass
    
    if years:
        plt.figure(figsize=(8,5))
        plt.hist(years, bins=10, edgecolor="black", color='lightgreen')
        plt.title("Release Year Distribution")
        plt.xlabel("Year")
        plt.ylabel("Count")
        plt.tight_layout()
        plt.show()
    else:
        print("No valid releaseDate data to visualize.")

def visualize_graph(g):
    """
    Visualize the RDF graph using NetworkX.
    """
    nx_graph = nx.Graph()
    for s, p, o in g:
        nx_graph.add_edge(str(s), str(o), label=str(p))
    
    if nx_graph.number_of_nodes() == 0:
        print("No data to visualize.")
        return
    
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(nx_graph, k=0.5)
    nx.draw(nx_graph, pos, with_labels=True, node_size=500, font_size=8, node_color="lightblue")
    edge_labels = nx.get_edge_attributes(nx_graph, 'label')
    nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels=edge_labels, font_size=7)
    plt.title("RDF Graph Visualization")
    plt.show()

# Optional EDA
# visualize_data(graph)
# visualize_graph(graph)



## 4. Système de Recommandation 

a) Utiliser SPARQL pour interroger le graphe RDF afin de créer un système de recommandation de films et/ou séries. Vous pouvez envisager différentes approches de recommandation, telles que la recommandation collaborative (en fonction des évaluations d'utilisateurs similaires) ou la recommandation basée sur le contenu (en fonction des genres, des acteurs, etc.) ou la recommandation basée sur les connaissances

b) Utiliser une IA (ChatGPT ou tout autre) pour répondre à cette question

In [78]:
# =============================================================================
# 4. CONTENT-BASED & AI-BASED RECOMMENDATIONS
#    Using multiple features: actors, directors, genres
# =============================================================================

# --- 4a) FEATURE-BASED (CONTENT-BASED) APPROACH ---
def recommend_feature_based(g, user_prefs):
    """
    Returns a list of films that match the user's preferred actors, directors, or genres.
    user_prefs = {
      "genres": [...],
      "directors": [...],
      "actors": [...]
    }
    """
    pref_genres = [pg.lower() for pg in user_prefs.get("genres", [])]
    pref_directors = [pd.lower() for pd in user_prefs.get("directors", [])]
    pref_actors = [pa.lower() for pa in user_prefs.get("actors", [])]

    recommended = set()

    for film in g.subjects(None, None):
        # Gather film's features
        film_genres = [str(o).lower() for o in g.objects(film, EX.genre)]
        film_directors = [str(o).lower() for o in g.objects(film, EX.director)]
        film_actors = [str(o).lower() for o in g.objects(film, EX.actor)]
        
        # Check matches
        match_genre = any(pg in fg for fg in film_genres for pg in pref_genres)
        match_director = any(pd in fd for fd in film_directors for pd in pref_directors)
        match_actor = any(pa in fa for fa in film_actors for pa in pref_actors)

        # If the film matches ANY of the user's preferences, add it
        if match_genre or match_director or match_actor:
            recommended.add(str(film))

    return list(recommended)

# --- 4b) AI-BASED (MULTI-FEATURE) APPROACH ---
def ai_recommend_multifeature(g, user_prefs):
    """
    AI-based recommendation that merges user preferences (genres, directors, actors)
    into a single text. Each film is also described by its title + actors + directors + genres.
    We use CountVectorizer + cosine similarity to rank films.
    """
    # Combine user preferences into a single text string
    user_genres = [g_.lower() for g_ in user_prefs.get("genres", [])]
    user_directors = [d_.lower() for d_ in user_prefs.get("directors", [])]
    user_actors = [a_.lower() for a_ in user_prefs.get("actors", [])]

    # e.g. "comedy horror spielberg atom_egoyan tom_hanks sandra_bullock"
    user_info = " ".join(user_genres + user_directors + user_actors).strip()
    if not user_info:
        print("No user preferences found. Returning empty recommendations.")
        return []

    # Build extended text for each film
    film_texts = {}
    for film in g.subjects(None, None):
        # Title
        labels = list(g.objects(film, RDFS.label))
        title_text = labels[0].lower() if labels else ""

        # Actors
        film_actors = [str(o).lower() for o in g.objects(film, EX.actor)]
        # Directors
        film_directors = [str(o).lower() for o in g.objects(film, EX.director)]
        # Genres
        film_genres = [str(o).lower() for o in g.objects(film, EX.genre)]

        # Combine
        # e.g. "cadet kelly tom hanks steven spielberg horror film"
        film_pieces = []
        if title_text:
            film_pieces.append(title_text)
        if film_actors:
            film_pieces.append(" ".join(film_actors))
        if film_directors:
            film_pieces.append(" ".join(film_directors))
        if film_genres:
            film_pieces.append(" ".join(film_genres))

        if film_pieces:
            film_uri_str = str(film)
            film_texts[film_uri_str] = " ".join(film_pieces)

    if not film_texts:
        print("No film data for AI-based recommendations.")
        return []

    # Vectorize user text vs. film text
    all_film_descriptions = list(film_texts.values())
    vectorizer = CountVectorizer()
    film_matrix = vectorizer.fit_transform(all_film_descriptions)
    user_vec = vectorizer.transform([user_info])

    # Cosine similarity
    sims = cosine_similarity(user_vec, film_matrix)[0]  # shape: [num_films]
    sorted_indices = np.argsort(sims)[::-1]

    film_uris = list(film_texts.keys())
    top_10 = [film_uris[i] for i in sorted_indices[:10]]
    return top_10


## 5. Calcul des Recommandations 

a) Utiliser SPARQL pour générer des requêtes de recommandation en fonction des préférences de l'utilisateur. Vous pouvez également utiliser des algorithmes d'apprentissage automatique pour améliorer les recommandations.

b) Utiliser une IA pour répondre à cette question

In [79]:
# =============================================================================
# 5. EVALUATION: MAP@k, NDCG@k
# =============================================================================
def evaluate_recommendations(ground_truth, predicted):
    """
    Evaluates recommendations using MAP@k and NDCG@k.
    
    ground_truth: list of URIs the user actually likes (favorite films).
    predicted: list of recommended film URIs.
    """
    if not predicted:
        return {"MAP@k": 0.0, "NDCG@k": 0.0}
    
    k = len(predicted)
    
    # MAP@k
    average_precision = 0.0
    relevant_found = 0
    for i, item in enumerate(predicted[:k]):
        if item in ground_truth:
            relevant_found += 1
            average_precision += relevant_found / (i + 1)
    map_k = average_precision / len(ground_truth) if ground_truth else 0.0

    # NDCG@k
    y_true = [1 if it in ground_truth else 0 for it in predicted[:k]]
    y_score = [1 / (idx + 1) for idx in range(k)]
    ndcg_k = ndcg_score([y_true], [y_score])

    return {"MAP@k": map_k, "NDCG@k": ndcg_k}


## 6. Évaluation  

a) Évaluez la qualité de vos recommandations en utilisant des mesures telles que  MAP@k (Mean Average Precision at k) ou NDCG@k (Normalized Discounted Cumulative Gain at k)

b) Comparer les recommandations que vous avez obtenues par rapport à celles générées par l'IA que vous aurez utilisée

In [80]:
# =============================================================================
# 6. COMPARE THE TWO APPROACHES: EXAMPLE USAGE
# =============================================================================
if __name__ == "__main__":
    user_prefs = {
        "genres": ["comedy", "horror"],
        "directors": ["spielberg", "atom_egoyan"],
        "actors": ["tom_hanks", "sandra_bullock"],
        "favorite_films": [
            # Make sure at least one of these URIs is in the recommended sets for nonzero scores
            "http://dbpedia.org/resource/Cadet_Kelly",  
            "http://dbpedia.org/resource/Cabin_by_the_Lake"
        ]
    }
    
    # --- Approach 1: Feature-based Content ---
    cb_recs = recommend_feature_based(graph, user_prefs)
    print("Feature-Based Content Recommendations:", cb_recs)
    
    # --- Approach 2: AI-based multi-feature ---
    ai_recs = ai_recommend_multifeature(graph, user_prefs)
    print("AI-Based Multi-Feature Recommendations:", ai_recs)
    
    # --- Evaluate ---
    ground_truth = user_prefs["favorite_films"]
    cb_scores = evaluate_recommendations(ground_truth, cb_recs)
    ai_scores = evaluate_recommendations(ground_truth, ai_recs)

    print("Feature-Based Content Evaluation:", cb_scores)
    print("AI-Based Evaluation:", ai_scores)

    if cb_scores["MAP@k"] > ai_scores["MAP@k"]:
        print("Feature-based content approach performed better (MAP@k).")
    elif cb_scores["MAP@k"] < ai_scores["MAP@k"]:
        print("AI-based multi-feature approach performed better (MAP@k).")
    else:
        print("They performed the same (MAP@k).")

Feature-Based Content Recommendations: ['http://dbpedia.org/resource/Cadet_Kelly', 'http://dbpedia.org/resource/California_Typewriter', 'http://dbpedia.org/resource/Calendar_(1993_film)', 'http://dbpedia.org/resource/Cabin_by_the_Lake', 'http://dbpedia.org/resource/Call_Me_Claus']
AI-Based Multi-Feature Recommendations: ['http://dbpedia.org/resource/Calendar_(1993_film)', 'http://dbpedia.org/resource/California_Typewriter', 'http://dbpedia.org/resource/Call_Girls_of_Rome', 'http://dbpedia.org/resource/Calamity_Anne,_Heroine', 'http://dbpedia.org/resource/Caitlin_Plays_Herself', 'http://dbpedia.org/resource/Call_Jane', 'http://dbpedia.org/resource/California_Straight_Ahead!', 'http://dbpedia.org/resource/Calendar_Girls_(2015_film)', "http://dbpedia.org/resource/Calamity_Anne's_Love_Affair", 'http://dbpedia.org/resource/Cagliostro_(1929_film)']
Feature-Based Content Evaluation: {'MAP@k': 0.75, 'NDCG@k': np.float64(0.8772153153380493)}
AI-Based Evaluation: {'MAP@k': 0.0, 'NDCG@k': np.floa

## 7. Rapport 

Vous allez rédiger un rapport de 5 pages max décrivant la modélisation RDF, les requêtes SPARQL, l'algorithme de recommandation et les résultats de l'évaluation. Vous spécifierez votre utilisation de l'IA dans le cadre de ce projet et ce que vous en pensez